<a href="https://colab.research.google.com/github/diegoflxgarcia/Pandas/blob/main/dataframes_e_banco_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criando, escrevendo, consultando e atualizando um banco de dados local

In [72]:
"""Vou utilizar uma versão inferior (a 2.0.20) do pacote sqlalchemy para ser
   compatível ao pacote Pandas"""
# !pip install --upgrade 'sqlalchemy<2.0'
# !pip install --upgrade 'sqlalchemy'
!pip show 'sqlalchemy'

Name: SQLAlchemy
Version: 1.4.49
Summary: Database Abstraction Library
Home-page: https://www.sqlalchemy.org
Author: Mike Bayer
Author-email: mike_mp@zzzcomputing.com
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: greenlet
Required-by: ipython-sql


## Preparando o banco de dados local a ser usado
O SQLalchemy fornece recursos para criar e gerenciar um banco de dados relacional local, com Python aliado ao SQL.

Vamos usa-lo junto com o Pandas para gerenciar dados tanto no banco de dados (database) e em DataFrames Pandas.

In [73]:
# Importando as bibliotecas, funções e classes necessárias
import pandas as pd
from sqlalchemy import create_engine, inspect, Table, MetaData

In [74]:
# Criando um banco de dados local, vamos usar o SQLite
engine = create_engine('sqlite:///:memory:')

## Vamos inserir nossos dados no nosso banco de dados local SQLite

Os dados em questão são informações de clientes de uma insituição financeira, importantes para realizar análises, e estão em um arquivo csv. Portanto vamos inseri-los no nosso banco, em um banco de dados fica mais fácil gerenciar, consultar e atualizar esses dados além do próprio armazenamento eficiente.



In [75]:
url = 'https://raw.githubusercontent.com/alura-cursos/Pandas/main/clientes_banco.csv'
df_clientes = pd.read_csv(url)
display(df_clientes.head())
print(f'{df_clientes.shape[0]} clientes')

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Categoria_de_renda,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,5008804,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
1,5008805,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
2,5008806,58,Ensino médio,Casado,2,Empregado,Segurança,3,112500.0,1,Casa/apartamento próprio
3,5008808,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
4,5008809,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio


438463 clientes


In [76]:
# Passando os dados para o nosso database local
df_clientes.to_sql('clientes', engine, index=False)

438463

In [77]:
# Verificando a tabela criada no nosso database
buscador = inspect(engine)
print(buscador.get_table_names())

['clientes']


## Consulta SQL
Vamos realizar agora uma consulta no nosso database com SQL e suas claúsulas, usando a função [pd.read_sql](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html). Essa consulta, por essa função, se traduz em um DataFrame Pandas.

[Link](https://support.microsoft.com/pt-br/office/acesso-sql-conceitos-b%C3%A1sicos-vocabul%C3%A1rio-e-sintaxe-444d0303-cde1-424e-9a74-e8dc3e460671#:~:text=SQL%20cl%C3%A1usulas&text=Cada%20cl%C3%A1usula%20executa%20uma%20fun%C3%A7%C3%A3o,as%20cl%C3%A1usulas%20SQL%20mais%20comuns.&text=Lista%20os%20campos%20que%20cont%C3%AAm%20dados%20de%20interesse.) sobre o SQL e sua sintaxe.

 - Separando as pessoas que estão com a categoria de renda preenchidas como "Empregado"

In [78]:
# Construindo a minha consulta
query = 'SELECT * FROM clientes WHERE Categoria_de_renda="Empregado"'

# A função pd.read_sql realiza consultas passando os parâmetros básicos: a consulta em SQL e o motor do banco de dados
df_clientes_empregados = pd.read_sql(query, engine)
display(df_clientes_empregados)

# Confirmando as categorias na coluna "Categoria_de_renda", queremos apenas "Empregado"
print(f'Na coluna "Categoria_de_renda" temos as seguintes categorias: {df_clientes_empregados.Categoria_de_renda.unique()}')

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Categoria_de_renda,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,5008804,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
1,5008805,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
2,5008806,58,Ensino médio,Casado,2,Empregado,Segurança,3,112500.0,1,Casa/apartamento próprio
3,5008815,46,Ensino superior,Casado,2,Empregado,Contabilidade,2,270000.0,1,Casa/apartamento próprio
4,5112956,46,Ensino superior,Casado,2,Empregado,Contabilidade,2,270000.0,1,Casa/apartamento próprio
...,...,...,...,...,...,...,...,...,...,...,...
226054,6837905,43,Ensino médio,Casado,3,Empregado,Outro,7,355050.0,1,Casa/apartamento próprio
226055,6837906,43,Ensino médio,Casado,3,Empregado,Outro,7,355050.0,1,Casa/apartamento próprio
226056,6839936,34,Ensino médio,Casado,3,Empregado,Construção Civil,5,135000.0,1,Casa/apartamento próprio
226057,6840222,43,Ensino médio,Solteiro,1,Empregado,Construção Civil,8,103500.0,0,Casa/apartamento próprio


Na coluna "Categoria_de_renda" temos as seguintes categorias: ['Empregado']


Escrevendo e salvando no nosso database nossa consulta dos clientes empregados com o método [to_sql](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)

In [79]:
# Salvando sem a coluna superflúa "Categoria_de_renda", já que só possui a categoria "Empregado"
df_clientes_empregados.drop(columns='Categoria_de_renda').to_sql('empregados', con=engine, index=False)

226059

Lendo a tabela inteira salva no database como DataFrame com a função [pd.read_sql_table](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_table.html)

In [80]:
df_clientes_empregados = pd.read_sql_table('empregados', con=engine)
display(df_clientes_empregados)

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,5008804,32,Ensino superior,União-estável,2,Outro,12,427500.0,1,Apartamento alugado
1,5008805,32,Ensino superior,União-estável,2,Outro,12,427500.0,1,Apartamento alugado
2,5008806,58,Ensino médio,Casado,2,Segurança,3,112500.0,1,Casa/apartamento próprio
3,5008815,46,Ensino superior,Casado,2,Contabilidade,2,270000.0,1,Casa/apartamento próprio
4,5112956,46,Ensino superior,Casado,2,Contabilidade,2,270000.0,1,Casa/apartamento próprio
...,...,...,...,...,...,...,...,...,...,...
226054,6837905,43,Ensino médio,Casado,3,Outro,7,355050.0,1,Casa/apartamento próprio
226055,6837906,43,Ensino médio,Casado,3,Outro,7,355050.0,1,Casa/apartamento próprio
226056,6839936,34,Ensino médio,Casado,3,Construção Civil,5,135000.0,1,Casa/apartamento próprio
226057,6840222,43,Ensino médio,Solteiro,1,Construção Civil,8,103500.0,0,Casa/apartamento próprio


## Atualizando nosso banco de dados
Vamos atualizar nosso database, removendo e adicionando dados e registros completos na tabela clientes dele (temos a "empregados" também).

In [81]:
# Primeiro vamos visualizar a tabela "clientes", usando sql (poderiamos usar a função read_sql_table, mas pretendo treinar o sql)
query = 'SELECT * FROM clientes'
display(pd.read_sql(query, engine))

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Categoria_de_renda,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,5008804,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
1,5008805,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
2,5008806,58,Ensino médio,Casado,2,Empregado,Segurança,3,112500.0,1,Casa/apartamento próprio
3,5008808,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
4,5008809,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
...,...,...,...,...,...,...,...,...,...,...,...
438458,6840104,62,Ensino médio,Divorciado,1,Pensionista,Outro,0,135000.0,0,Casa/apartamento próprio
438459,6840222,43,Ensino médio,Solteiro,1,Empregado,Construção Civil,8,103500.0,0,Casa/apartamento próprio
438460,6841878,22,Ensino superior,Solteiro,1,Associado comercial,Vendas,1,54000.0,0,Mora com os pais
438461,6842765,59,Ensino médio,Casado,2,Pensionista,Outro,0,72000.0,0,Casa/apartamento próprio


- Deletando o cliente de ID 5008805, pois esse cliente teve a conta cancelada

In [82]:
# Clausula DELETE
query = 'DELETE FROM clientes WHERE ID_Cliente=5008805'
# Método connect para acessar e alterar o database e a clausula python with para garantir o fechamento do database depois da alteração
with engine.connect() as conn:
  conn.execute(query)
# Confirmando...
display(pd.read_sql_table('clientes', engine))
print(f'O cliente de ID 5008805 ainda está no banco de dados de clientes? {5008805 in pd.read_sql_table("clientes", engine, columns=["ID_Cliente"])}')

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Categoria_de_renda,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,5008804,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
1,5008806,58,Ensino médio,Casado,2,Empregado,Segurança,3,112500.0,1,Casa/apartamento próprio
2,5008808,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
3,5008809,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
4,5008810,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
...,...,...,...,...,...,...,...,...,...,...,...
438457,6840104,62,Ensino médio,Divorciado,1,Pensionista,Outro,0,135000.0,0,Casa/apartamento próprio
438458,6840222,43,Ensino médio,Solteiro,1,Empregado,Construção Civil,8,103500.0,0,Casa/apartamento próprio
438459,6841878,22,Ensino superior,Solteiro,1,Associado comercial,Vendas,1,54000.0,0,Mora com os pais
438460,6842765,59,Ensino médio,Casado,2,Pensionista,Outro,0,72000.0,0,Casa/apartamento próprio


O cliente de ID 5008805 ainda está no banco de dados de clientes? False


- Atualizando grau de escolaridade do cliente de ID 5008808

In [83]:
# Clausula UPDATE e SET
query = 'UPDATE clientes SET Grau_escolaridade="Ensino superior" WHERE ID_Cliente=5008808'
with engine.connect() as conn:
  conn.execute(query)
# Confirmando...
display(pd.read_sql_table('clientes', engine))
query = 'SELECT ID_Cliente, Grau_escolaridade FROM clientes WHERE ID_Cliente=5008808'
display(pd.read_sql(query, engine))

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Categoria_de_renda,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,5008804,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
1,5008806,58,Ensino médio,Casado,2,Empregado,Segurança,3,112500.0,1,Casa/apartamento próprio
2,5008808,52,Ensino superior,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
3,5008809,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
4,5008810,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
...,...,...,...,...,...,...,...,...,...,...,...
438457,6840104,62,Ensino médio,Divorciado,1,Pensionista,Outro,0,135000.0,0,Casa/apartamento próprio
438458,6840222,43,Ensino médio,Solteiro,1,Empregado,Construção Civil,8,103500.0,0,Casa/apartamento próprio
438459,6841878,22,Ensino superior,Solteiro,1,Associado comercial,Vendas,1,54000.0,0,Mora com os pais
438460,6842765,59,Ensino médio,Casado,2,Pensionista,Outro,0,72000.0,0,Casa/apartamento próprio


,ID_Cliente,Grau_escolaridade
0,5008808,Ensino superior


- Atualizando o registro do cliente de ID 6840104 que teve o rendimento anual alterado para 300000.

In [84]:
query = 'UPDATE clientes SET Rendimento_anual=300000 WHERE ID_Cliente=6840104'
with engine.connect() as conn:
  conn.execute(query)
# Confirmando...
query = 'SELECT ID_Cliente, Rendimento_anual FROM clientes WHERE ID_Cliente=6840104'
display(pd.read_sql(query, engine))

,ID_Cliente,Rendimento_anual
0,6840104,300000.0


- Excluindo o registro do cliente de ID 5008809, pois esse cliente não possui mais conta na instituição financeira

In [85]:
query = 'DELETE FROM clientes WHERE ID_Cliente=5008809'
with engine.connect() as conn:
  conn.execute(query)
# Confirmando...
print(f'O cliente de ID 5008809 ainda está no banco de dados de clientes? {5008809 in pd.read_sql_table("clientes", engine, columns=["ID_Cliente"])}')

O cliente de ID 5008809 ainda está no banco de dados de clientes? False


- Criando um novo registro de cliente seguindo as especificações abaixo:
 - ID_Cliente: 6850985
 - Idade: 33
 - Grau_escolaridade: Doutorado
 - Estado_civil: Solteiro
 - Tamanho_familia: 1
 - Categoria_de_renda: Empregado
 - Ocupacao: TI
 - Anos_empregado: 2
 - Rendimento_anual: 290000
 - Tem_carro: 0
 - Moradia: Casa/apartamento próprio

In [86]:
colunas = 'ID_Cliente, Idade, Grau_escolaridade, Estado_civil, Tamanho_familia, Categoria_de_renda, Ocupacao, Anos_empregado, Rendimento_anual, Tem_carro, Moradia'
valores = '6850985, 33, "Doutorado", "Solteiro", 1, "Empregado", "TI", 2, 290000, 0, "Casa/apartamento próprio"'
query = f'INSERT INTO clientes ({colunas}) VALUES ({valores})'
with engine.connect() as conn:
  conn.execute(query)
# Confirmando...
query = 'SELECT * FROM clientes WHERE ID_Cliente=6850985'
display(pd.read_sql(query, engine))

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Categoria_de_renda,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,6850985,33,Doutorado,Solteiro,1,Empregado,TI,2,290000.0,0,Casa/apartamento próprio
